# Analysis of CIA World Factbook

We will use a data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/) that contains various statistics about all the countries on Earth.

## 1.0 Setup & Exploration

In [47]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [48]:
%%sql
/* cheking db */

SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [49]:
# import sqlite3
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from IPython.display import display

# # Create the connection
# con = sqlite3.connect(r'factbook.db')

# # create the dataframe from a query
# facts = pd.read_sql_query("SELECT * FROM facts", con)

# display(facts.shape)
# facts.head(10)


Explanation of the columns:
- `name` -- Name of the country
- `area` -- Total area of the country, land and water (in km^2)
- `area_land` -- Land area in km^2
- `area_water` -- Water area in km^2
- `population` -- The country's population
- `population_growth` -- Population growth as a percentage
- `birth_rate` -- Number of births per year per 1000 people
- `death_rate` -- Number of deaths per year per 1000 people
- `migration_rate` -- Difference between people entering and leaving the country per year per 1000 people

In [50]:
%%sql
SELECT MIN(population),
       MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts; 

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


According to the query above, there are countries with `0` population and population growth. There is also a country with a population over `7.2`bn. Let's investigate these values.

In [51]:
%%sql
SELECT *
  FROM facts
WHERE population = (SELECT MIN(population)
                      FROM facts
                    );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [52]:
%%sql
SELECT *
  FROM facts
WHERE population = (SELECT MAX(population)
                      FROM facts
                    );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The 'country' with no population is Antarctica, a polar region with no countries or permanent habitation outside of research stations.
The db also contains an entry for `World` which is the 'country' with a population of `7.2`bn.

Because of the `World` entry we will need to re-calculate the statistics we did before, with this entry excluded.

In [53]:
%%sql
SELECT MIN(population),
       MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name != 'World'

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [54]:
%%sql
SELECT *
  FROM facts
WHERE population = (SELECT MAX(population)
                      FROM facts
                     WHERE name != 'World'
                    );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


With `World` removed the highest population is `1.3`bn, which corresponds to China.

In [55]:
%%sql
SELECT ROUND(AVG(population) / 1000000, 2) AS 'Avg. Population (in MM)',
       ROUND(AVG(area), 2) AS 'Avg. Area (in km^2)'
  FROM facts;

 * sqlite:///factbook.db
Done.


Avg. Population (in MM),Avg. Area (in km^2)
62.09,555093.55


We'll run a query to find out what countries have populations *above* the average and also have total areas *below* the average.

In [56]:
%%sql
SELECT name, area, area_land, area_water,
       ROUND(population / 1000000, 2) AS 'population (in MM)'
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts                
                    ) 
         AND area < (SELECT AVG(area)
                       FROM facts
                    )
 ORDER BY population DESC


 * sqlite:///factbook.db
Done.


name,area,area_land,area_water,population (in MM)
Bangladesh,148460,130170,18290,168.0
Japan,377915,364485,13430,126.0
Philippines,300000,298170,1830,100.0
Vietnam,331210,310070,21140,94.0
Germany,357022,348672,8350,80.0
Thailand,513120,510890,2230,67.0
United Kingdom,243610,241930,1680,64.0


Due to large populations and small land areas these countries are very population dense.

- Which country has the most people? Which country has the highest growth rate?
- Which countries have the highest ratios of water to land? Which countries have more water than land?
- Which countries will add the most people to their populations next year?
- Which countries have a higher death rate than birth rate?
- Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?